2025ML_HW1_sample.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1OkvJ8FC8naaNZJWrgsv7-WjqGqFM1gd9

# **2025 ML FALL HW1: PM2.5 Prediction (Regression)**

Author: MLTAs

Methods:
* Training with all data
* Optimizer: RMSProp (default)
* TODOs:
  - Complete the `valid()` function
  - Tune the hyperparameters in `train_config`
  - Implement 2nd-order polynomial regression model (without interaction terms) in `minibatch_2()`
  - Implement feature normalization in `normalize_train_data()`
  - Feature selection

# **Import Some Packages**

In [18]:
import numpy as np
import csv
import math
import pandas as pd
import os

from tqdm import tqdm
from argparse import Namespace

# **Fix random seed**


This is for the reproduction of your result. **DO NOT modify this secton!**

In [19]:
seed = 9487
np.random.seed(seed)

#**Download training data**

In [20]:
!gdown --id "1Hfzrcm69QwdFvdeF0uASoQlcVxKw_hHy" --output "train.csv"
!gdown --id '155N6fzI7vAFzHAGdy6jkaWIksWH6Y1G2' --output "test.csv"

# Incase the links above die, you can use the following instead.
#!gdown --id '11abE854Eyv4BA7qt5k8r_80sJ3KuOQUN' --output "train.csv"
#!gdown --id '1uod-Z4ztluXnuHtgUbm39nMudUKqXHMl' --output "test.csv"

# If the data is still missing, you can manually download it from kaggle, and upload the files under /content

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Hfzrcm69QwdFvdeF0uASoQlcVxKw_hHy
To: /content/train.csv
100% 324k/324k [00:00<00:00, 101MB/s]
Traceback (most recent call last):
  File "/usr/local/bin/gdown", line 4, in <module>
    from gdown.__main__ import main
  File "/usr/local/lib/python3.12/dist-packages/gdown/__init__.py", line 6, in <module>
    from .cached_download import cached_download
object address  : 0x7ece020e6c20
object refcount : 3
object type     : 0xa274e0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
  File "/usr/local/lib/python3.12/dist-packages/gdown/cached_download.py", line 10, in <module>
^C


In [22]:
def valid(x, y):
    # TODO: Try to filter out extreme values.
    #  ex: If PM2.5 > 100, then we don't use the data to train (return False), otherwise return True,

    for i in range(x.shape[0]):
        IQR = np.percentile(x[i, :], 75) - np.percentile(x[i, :], 25)
        # print(x[i, :])
        # print(IQR)
        for j in range(x.shape[1]):
            if (x[i, j] > np.percentile(x[i, :], 75) + 3 * IQR) or (x[i, j] < np.percentile(x[i, :], 25) - 3 * IQR):
                return False

    if y > 50:
        return False

    return True


# Create your dataset
def parse2train(data, feats):

    x = []
    y = []
    print (data.shape)

    # Use data #0~#7 to predict #8 => Total data length should be decresased by 8.
    total_length = data.shape[1] - 8

    for i in tqdm(range(total_length)):
        x_tmp = data[feats, i:i+8] # Use data #0~#7 to predict #8, data #1~#8 to predict #9, etc.
        y_tmp = data[-1, i+8] # last column of (i+8)th row: PM2.5
        # print(x_tmp.shape)

        # Filter out extreme values to train.
        if valid(x_tmp, y_tmp):
            x.append(x_tmp.reshape(-1,))
            y.append(y_tmp)
    # x.shape: (n, 15, 8)
    # y.shape: (n, 1)
    x = np.array(x)
    y = np.array(y)

    return x,y

#**Gradient descent**
###**RMSProp**
1. $v_t=\beta \cdot v_{t-1} + (1-\beta)(\nabla w_t)^2$
2. $w_{t+1}=w_t - \frac{\eta}{\sqrt{(v_t)}+\epsilon}\nabla w_t$




* This is our gradient descent algorithm. RMSProp was implemented in `minibatch()`.
* You can implement other algorithm, such as SGD or other gradient descent variants listed below, which may (or may not) improve performance.
* However, **modules like sklearn and pytorch are not allowed!!!**
* Ref:
  - Prof. G. Hinton's lecture: https://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf
  - Prof. Hung-Yi Lee's video: https://youtu.be/HYUXEeh3kwY?si=RtLjSj51WK1pmz87

###**Adam (RMSProp + Momemtum)**
* Ref:
  - Paper: https://arxiv.org/pdf/1412.6980
  - Prof. Hung-Yi Lee's video: https://youtu.be/HYUXEeh3kwY?si=RtLjSj51WK1pmz87

###**AdamW (Adam with decoupled weight decay)**
* Ref:
  - Paper: https://arxiv.org/pdf/1711.05101


In [23]:
def minibatch(x, y, config):
    # Randomize the data in minibatch
    index = np.arange(x.shape[0])
    np.random.shuffle(index)
    x = x[index]
    y = y[index]

    # Initialization
    batch_size = config.batch_size
    lr = config.lr
    epoch = config.epoch
    decay_rate = config.decay_rate
    epsilon = 1e-8

    # Linear regression: only contains two parameters (w, b).
    w = np.full(x[0].shape, 0.1).reshape(-1, 1)
    bias = 0.1

    # Optimizer states
    cache_w = np.zeros_like(w)
    cache_b = 0.0

    # Training loop
    for num in range(epoch):
        for b in range(int(x.shape[0] / batch_size)):
            x_batch = x[b * batch_size:(b + 1) * batch_size]
            y_batch = y[b * batch_size:(b + 1) * batch_size].reshape(-1, 1)

            # Prediction of linear regression
            pred = np.dot(x_batch, w) + bias

            # Loss
            loss = y_batch - pred

            # Compute gradient
            g_t = np.dot(x_batch.transpose(), loss) * (-2)
            g_t_b = loss.sum(axis=0) * (-2)

            # Update cache
            cache_w = decay_rate * cache_w + (1 - decay_rate) * g_t**2
            cache_b = decay_rate * cache_b + (1 - decay_rate) * g_t_b**2

            # Update weight & bias
            w -= lr * g_t / (np.sqrt(cache_w) + epsilon)
            bias -= lr * g_t_b / (np.sqrt(cache_b) + epsilon)


    return w, bias

# TODO: Implement 2-nd polynomial regression version for the report.
def minibatch_2(x, y, config):
    # 打亂資料
    index = np.arange(x.shape[0])
    np.random.shuffle(index)
    x = x[index]
    y = y[index]

    x_poly = polynomial_features(x, degree=2)

    # Initialization
    batch_size = config.batch_size
    lr = config.lr
    epoch = config.epoch
    decay_rate = config.decay_rate
    epsilon = 1e-8

    w = np.full((x_poly.shape[1], 1), 0.1)  # 權重
    bias = 0.1

    # Optimizer states
    cache_w = np.zeros_like(w)
    cache_b = 0.0

    # Training loop
    for num in range(epoch):
        for b in range(int(x_poly.shape[0] / batch_size)):
            x_batch = x_poly[b * batch_size:(b + 1) * batch_size]
            y_batch = y[b * batch_size:(b + 1) * batch_size].reshape(-1, 1)

            # Prediction
            pred = np.dot(x_batch, w) + bias

            # Loss
            loss = y_batch - pred

            # Gradient
            g_t = np.dot(x_batch.T, loss) * (-2)
            g_t_b = loss.sum(axis=0) * (-2)

            # Update cache (RMSProp)
            cache_w = decay_rate * cache_w + (1 - decay_rate) * g_t**2
            cache_b = decay_rate * cache_b + (1 - decay_rate) * g_t_b**2

            # Update params
            w -= lr * g_t / (np.sqrt(cache_w) + epsilon)
            bias -= lr * g_t_b / (np.sqrt(cache_b) + epsilon)

    return w, bias

def polynomial_features(X, degree=2):

    n, d = X.shape
    features = [X]  # 一階
    for j in range(d):
        features.append(X[:, j:j+1] ** 2)  # 平方
    return np.hstack(features)

In [24]:
# TODO: Tune the config to boost your performance.
train_config = Namespace(
    batch_size = 100,
    lr = 0.001,
    epoch = 200,
    decay_rate = 0.9
)

# **Training your regression model**

In [25]:
train_df = pd.read_csv("/content/train.csv")
train_df

,AMB_TEMP,CO,NO,NO2,NOx,O3,PM10,WS_HR,RAINFALL,RH,SO2,WD_HR,WIND_DIREC,WIND_SPEED,PM2.5
0,10.8,0.32,1.7,8.6,10.3,22.9,21,0.6,0.0,71,1.9,172,171,0.6,15
1,10.8,0.27,1.6,6.2,7.8,23.8,20,1.4,0.0,71,1.7,161,129,1.8,13
2,11.0,0.25,0.9,5.4,6.3,27.4,21,0.8,0.0,68,1.6,152,147,1.5,12
3,11.0,0.23,0.7,3.1,3.8,29.5,21,1.8,0.0,68,1.6,138,145,1.7,9
4,11.3,0.22,0.8,2.9,3.8,30.7,16,1.9,0.0,67,1.6,140,139,1.7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5769,29.0,0.41,1.2,14.0,15.3,23.0,21,1.4,0.0,74,2.8,149,168,2.0,14
5770,28.2,0.33,1.7,11.7,13.5,19.5,23,2.1,0.0,78,2.3,187,179,2.5,15
5771,28.0,0.29,1.3,9.1,10.4,17.6,17,1.5,0.0,78,2.0,173,200,1.5,13
5772,28.0,0.27,1.4,9.5,11.0,15.4,17,1.1,0.0,75,1.8,171,135,0.9,10


In [26]:
# TODO: Normalize each column (except PM2.5) for the report (use z-score normalization)
def normalize_train_data(df):
    """
    Steps:
    1. For each column (except PM2.5): calculate mean and std
    2. Apply standardization: (column - mean) / std
    3. Store normalization parameters for later use on test data

    Returns:
        normalized_df: DataFrame with normalized features
        norm_params: Dict with {'column': {'mean': X, 'std': Y}}

    Hint: Loop through data.columns, skip PM2.5
    """
    # Your implementation here
    norm_params = {}
    for column in df.columns:
        if column == 'PM2.5':
            continue
        mean = df[column].mean()
        std = df[column].std()
        norm_params[column] = {'mean': mean, 'std': std}
        df[column] = (df[column] - mean) / std

    return df, norm_params

#**feature selection**

In [27]:
# Choose your features to train.
# Hint:
# 1. You can select more than one feature.
# 2. You should select "good" features.

# TODO: Carefully justify which feature should be chosen.

train_df_X = train_df.drop(train_df.index[-1]).values
train_df_y = train_df.drop(train_df.index[0])['PM2.5'].values

def ols_fit(X, y):
    X = np.asarray(X, dtype=float)
    y = np.asarray(y, dtype=float)
    beta = np.linalg.inv(X.T @ X) @ X.T @ y
    y_pred = X @ beta
    residuals = y - y_pred
    sse = np.sum(residuals**2)
    return beta, sse

def aic(sse, n, k):
    return n * np.log(sse / n) + 2 * k

def stepwise_selection(X, y):
    n, p = X.shape
    selected = []
    remaining = list(range(p))
    current_aic = np.inf

    while True:
        changed = False

        # ---------- Forward step ----------
        aic_candidates = []
        for feature in remaining:
            features_to_test = selected + [feature]
            X_test = np.c_[np.ones(n), X[:, features_to_test]]
            _, sse = ols_fit(X_test, y)
            k = len(features_to_test) + 1
            aic_val = aic(sse, n, k)
            aic_candidates.append((aic_val, feature))

        if aic_candidates:
            aic_candidates.sort()
            best_aic, best_feature = aic_candidates[0]

            if best_aic < current_aic:
                current_aic = best_aic
                selected.append(best_feature)
                remaining.remove(best_feature)
                changed = True
                print(f"Forward: 加入特徵 {best_feature}, AIC={best_aic:.2f}")

        # ---------- Backward step ----------
        if len(selected) > 1:
            aic_candidates = []
            for feature in selected:
                features_to_test = [f for f in selected if f != feature]
                X_test = np.c_[np.ones(n), X[:, features_to_test]]
                _, sse = ols_fit(X_test, y)
                k = len(features_to_test) + 1
                aic_val = aic(sse, n, k)
                aic_candidates.append((aic_val, feature))

            aic_candidates.sort()
            best_aic, worst_feature = aic_candidates[0]

            if best_aic < current_aic:
                current_aic = best_aic
                selected.remove(worst_feature)
                remaining.append(worst_feature)
                changed = True
                print(f"Backward: 移除特徵 {worst_feature}, AIC={best_aic:.2f}")

        if not changed:
            break

    return selected

feats = stepwise_selection(train_df_X, train_df_y)

Forward: 加入特徵 14, AIC=15188.96
Forward: 加入特徵 6, AIC=15059.42
Forward: 加入特徵 1, AIC=14888.07
Forward: 加入特徵 11, AIC=14807.43
Forward: 加入特徵 4, AIC=14776.61
Forward: 加入特徵 8, AIC=14752.17
Forward: 加入特徵 5, AIC=14732.52
Forward: 加入特徵 13, AIC=14716.84
Forward: 加入特徵 10, AIC=14704.32
Forward: 加入特徵 9, AIC=14696.52
Forward: 加入特徵 12, AIC=14691.84


In [28]:
# Training data preprocessing
def train_processing(train_df, norm=False):
    """Process training train_df with optional normalization"""

    if norm:
        # Normalize training data and save parameters (mean & std)
        data_norm, norm_params = normalize_train_data(train_df)
        data_values = data_norm.values
    else:
        # Use raw training data
        data_values = train_df.values
        norm_params = None

    # Common processing steps
    train_data = np.transpose(np.array(np.float64(data_values)))
    train_x, train_y = parse2train(train_data, feats)

    return train_x, train_y, norm_params

train_x, train_y, norm_params = train_processing(train_df, norm=False)

# Train your regression model
w, bias = minibatch(train_x, train_y, train_config)
print(w, bias)

(15, 5774)


100%|██████████| 5766/5766 [01:10<00:00, 81.90it/s]


[[ 2.90155737e-02]
 [ 2.52235002e-02]
 [ 4.35066480e-02]
 [ 4.42675999e-02]
 [ 2.18789874e-02]
 [ 7.62589603e-02]
 [ 1.58127013e-01]
 [ 4.63245150e-01]
 [-1.16571494e-02]
 [-3.42147553e-03]
 [-2.07194747e-02]
 [-2.42034466e-02]
 [-1.53182034e-02]
 [-3.78246504e-02]
 [-2.38508795e-02]
 [ 1.35805506e-01]
 [ 7.98609375e-02]
 [ 1.07183516e-01]
 [ 1.06260334e-01]
 [ 1.06372217e-01]
 [ 1.00624359e-01]
 [ 1.09105068e-01]
 [ 1.54407546e-01]
 [ 2.90203745e-01]
 [-8.14253535e-04]
 [-6.28184132e-04]
 [ 2.10390508e-03]
 [-4.92097299e-04]
 [ 1.56673751e-03]
 [-5.16430050e-03]
 [-4.08543291e-04]
 [ 6.41990792e-04]
 [-7.95871394e-03]
 [ 1.34636578e-02]
 [-1.87666748e-02]
 [ 5.72130871e-03]
 [-2.27525526e-02]
 [-2.92192126e-02]
 [-1.63457964e-02]
 [ 8.08370080e-02]
 [ 5.96567409e-02]
 [-7.49822274e-02]
 [-5.26975082e-02]
 [ 3.71073747e-02]
 [-3.71608151e-02]
 [-8.78170571e-02]
 [ 5.88388267e-02]
 [-2.13055632e-01]
 [-3.87451751e-03]
 [-1.50065461e-03]
 [ 1.58251526e-02]
 [-5.25583818e-03]
 [-1.2569750

# **Testing:**

In [29]:
def parse2test(data, feats):
    x = []
    for i in range(90):
        x_tmp = data[feats, 8*i: 8*i+8]
        x.append(x_tmp.reshape(-1,))
    # x.shape: (n, 15, 8)
    x = np.array(x)
    return x

def normalize_test_data(df, norm_params):
    data_norm = df.copy()

    for col, params in norm_params.items():
        if col in df.columns:
            data_norm[col] = (df[col] - params['mean']) / params['std']

    return data_norm

test_df = pd.read_csv('/content/test.csv')
# test_df.info()

# Testing data preprocessing
def test_processing(test_df, norm=False, norm_params=norm_params):
    if norm:
        if norm_params is None:
            raise ValueError("norm_params required when norm=True")

        # Apply training normalization parameters to testing data
        data_norm = normalize_test_data(test_df, norm_params)
        data_values = data_norm.values
    else:
        # Use raw testing data
        data_values = test_df.values

    # Common processing steps
    test_data = np.transpose(np.array(np.float64(data_values)))
    test_x = parse2test(test_data, feats)

    return test_x

test_x = test_processing(test_df, norm=False, norm_params=norm_params)

#**Write result as .csv**

In [30]:
with open('/content/stepwise_e200_lr1e-3_wovalid.csv', 'w', newline='') as csvf:
    writer = csv.writer(csvf)
    writer.writerow(['Id','Predicted'])
    pred_y = []
    for i in range(int(test_x.shape[0])):
        # Prediction of linear regression
        prediction = (np.dot(np.reshape(w,-1),test_x[i]) + bias)[0]
        pred_y.append(prediction)
        writer.writerow([i, prediction])

